In [0]:
#DOWNLOAD & EXTRACT FILE
import zipfile
import requests
from pyspark.sql.types import *
import random
from pyspark.sql.functions import lit

url='https://www.briandunning.com/sample-data/us-500.zip'
req = requests.get(url)

filename = url.split('/')[-1]
 
# Writing the file to the local file system
with open(filename,'wb') as output_file:
    output_file.write(req.content)
print('Downloading Completed')


with zipfile.ZipFile("/databricks/driver/us-500.zip", 'r') as zip_ref:
    zip_ref.extractall("/databricks/driver")

Downloading Completed

In [0]:
#source dataframe and list of 50000 numbers
sourceDf=spark.read.format("csv").option("header",True).load("file:/databricks/driver/us-500.csv")
# sourceDf.show(10,False)
sourceDf.registerTempTable("test")

l1=[i for i in range(1,50001)]
fullSeqDf=spark.createDataFrame(l1, IntegerType())
fullSeqDf.registerTempTable("seq")
fullSeqDf.show()

+-----+
value|
+-----+
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
 9|
 10|
 11|
 12|
 13|
 14|
 15|
 16|
 17|
 18|
 19|
 20|
+-----+
only showing top 20 rows

In [0]:
#dataset prepare
names=spark.sql("""
with names as (
  SELECT first_name as name FROM test
  ),
names_with_nums as (
  select row_number() OVER (ORDER BY n1.name,n2.name) as id,
    n1.name as first_name,
    n2.name as last_name,
    lower(n1.name)||'.'||lower(n2.name)||"@gmail.com" as email_id
  FROM names as n1
  CROSS JOIN names as n2
  WHERE n1.name!=n2.name)
select * FROM names_with_nums
WHERE id<=50001
""")

names.registerTempTable("names")
# names.show()

locations=spark.sql("""
WITH loc_prepare AS (
SELECT DISTINCT city,county,state,zip FROM test )
SELECT (row_number() over (order by city,county,state,zip))-1 loc_id,*
FROM loc_prepare
""")

locations.registerTempTable("locations")
# locations.show()

companies=spark.sql("""
WITH com_prepare AS (
SELECT DISTINCT company_name,web FROM test )
SELECT (row_number() over (order by company_name,web))-1 loc_id,*
FROM com_prepare
""")

companies.registerTempTable("companies")
# companies.show()

addresses=spark.sql("""
with names as (
  SELECT last_name as name FROM test
  ),
names_with_nums as (
  select row_number() OVER (ORDER BY n1.name,n2.name) as id,
    n1.name as first_name,
    n2.name as last_name
  FROM names as n1
  CROSS JOIN names as n2
  WHERE n1.name!=n2.name
  ),
addresses as (
  select id,
  id||" "||first_name||" St. "|| last_name  as address FROM names_with_nums
  WHERE id<=50001
  )
SELECT * FROM addresses
""")

addresses.registerTempTable("addresses")
addresses.show()

def formPh(ph):
  strPh=str(ph)
  return f"{strPh[:3]}-{strPh[3:6]}-{strPh[6:10]}"

ph1=[formPh(9842268754+i) for i in range(0,50001)]
phone1=spark.createDataFrame(ph1, StringType())
phone1.registerTempTable("ph1")
phoneNum1=spark.sql("""SELECT row_number() OVER (ORDER BY value) as id,value no FROM ph1""")
phoneNum1.registerTempTable("phoneNum1")

ph2=[formPh(8842468354+i) for i in range(0,50001)]
phone2=spark.createDataFrame(ph2, StringType())
phone2.registerTempTable("ph2")
phoneNum2=spark.sql("""SELECT row_number() OVER (ORDER BY value) as id,value no FROM ph2""")
phoneNum2.registerTempTable("phoneNum2")

+---+--------------------+
 id| address|
+---+--------------------+
 1| 1 Abdallah St. Acey|
 2|2 Abdallah St. Acuff|
 3|3 Abdallah St. Adkin|
 4|4 Abdallah St. Ag...|
 5| 5 Abdallah St. Ahle|
 6|6 Abdallah St. Al...|
 7|7 Abdallah St. Am...|
 8|8 Abdallah St. Amyot|
 9|9 Abdallah St. An...|
 10|10 Abdallah St. A...|
 11|11 Abdallah St. A...|
 12|12 Abdallah St. A...|
 13|13 Abdallah St. A...|
 14|14 Abdallah St. A...|
 15|15 Abdallah St. A...|
 16|16 Abdallah St. Asar|
 17|17 Abdallah St. A...|
 18|18 Abdallah St. A...|
 19|19 Abdallah St. B...|
 20|20 Abdallah St. B...|
+---+--------------------+
only showing top 20 rows

In [0]:
#form dateset

employees=spark.sql("""

with emp as (
  SELECT row_number() over (order by first_name,last_name) as empId,* FROM test
),
seqn as (
  SELECT value,int(RAND()*10000) r_val
  FROM seq 
),
emp_blow as (
  SELECT 
    s.value as emp_id,
    coalesce(t.first_name,n.first_name) as first_name,
    coalesce(t.last_name,n.last_name) as last_name,
    coalesce(t.company_name,c.company_name) as company_name,
    coalesce(t.address,a.address) as address,
    coalesce(t.city,l.city) as city,
    coalesce(t.county,l.county) as county,
    coalesce(t.state,l.state) as state,
    coalesce(t.zip,l.zip) as zip,
    coalesce(t.phone1,p1.no) as phone1,
    coalesce(t.phone2,p2.no) as phone2,
    coalesce(t.email,n.email_id) as email,
    coalesce(t.web,c.web) as web
  FROM seqn s
  LEFT JOIN emp t ON s.value=t.empId
  LEFT JOIN names n ON s.value=n.id
  LEFT JOIN locations l ON mod(s.value*s.r_val,451)=l.loc_id
  LEFT JOIN companies c ON mod(s.value*s.r_val,499)=c.loc_id
  LEFT JOIN addresses a ON s.value=a.id
  LEFT JOIN phoneNum1 p1 ON s.value=p1.id
  LEFT JOIN phoneNum2 p2 ON s.value=p2.id)
select * from emp_blow""")
employees.persist()
employees.registerTempTable("employees")

In [0]:
#Create a dataframe of CityEmployeeDensity, the 1st city will be the one with maxium number of employees

CityEmployeeDensity=spark.sql(""" WITH emp_counts as (
   SELECT city,COUNT(*) as emp_count
   FROM employees
   GROUP BY city
 )
 SELECT city,emp_count,dense_rank() over (order by emp_count desc) as vaccination_order
 FROM emp_counts""")
CityEmployeeDensity.persist()
CityEmployeeDensity.show()
CityEmployeeDensity.registerTempTable("CityEmployeeDensity")


  


+------------+---------+-----------------+
 city|emp_count|vaccination_order|
+------------+---------+-----------------+
 New York| 1061| 1|
Philadelphia| 814| 2|
 Chicago| 741| 3|
 Miami| 698| 4|
 Orlando| 622| 5|
 Milwaukee| 567| 6|
 Providence| 548| 7|
 Denver| 489| 8|
Indianapolis| 480| 9|
 Baltimore| 471| 10|
 Los Angeles| 461| 11|
 Aberdeen| 416| 12|
 Brooklyn| 410| 13|
 Atlanta| 407| 14|
 Phoenix| 366| 15|
 Gardena| 335| 16|
 Bridgeport| 325| 17|
 New Orleans| 321| 18|
 San Diego| 317| 19|
 Boise| 315| 20|
+------------+---------+-----------------+
only showing top 20 rows

In [0]:
VaccinationDrivePlan=employees.withColumn("Sequence",lit(-1))
VaccinationDrivePlan.show()
VaccinationDrivePlan.registerTempTable("VaccinationDrivePlan")

+------+----------+----------+--------------------+--------------------+---------------+-----------+-----+-----+------------+------------+--------------------+--------------------+--------+
emp_id|first_name| last_name| company_name| address| city| county|state| zip| phone1| phone2| email| web|Sequence|
+------+----------+----------+--------------------+--------------------+---------------+-----------+-----+-----+------------+------------+--------------------+--------------------+--------+
 148| Ernie| Stenseth| Knwz Newsradio| 45 E Liberty St|Ridgefield Park| Bergen| NJ|07660|201-709-6245|201-387-9093|ernie_stenseth@ao...|http://www.knwzne...| -1|
 463| Tonette| Wenner|Northwest Publishing| 4545 Courthouse Rd| Westbury| Nassau| NY|11590|516-968-6051|516-333-4861| twenner@aol.com|http://www.northw...| -1|
 471| Valentin| Klimek|Schmid, Gayanne K...| 137 Pioneer Way| Chicago| Cook| IL|60604|312-303-5453|312-512-2338| vklimek@klimek.org|http://www.schmid...| -1|
 496| Yolando| Luczki|Dal Tile Corporation| 422 E 21st St| Syracuse| Onondaga| NY|13214|315-304-4759|315-640-6357| yolando@cox.net|http://www.daltil...| -1|
 833| Adelina| Maryann| Kleensteel|833 Acey St. Pagl...| Amarillo| Randall| TX|79109|984-226-9586|884-246-9186|adelina.maryann@g...|http://www.kleens...| -1|
 1088| Adell|Christiane| Magnuson|1088 Acuff St. Co...| Abilene| Dickinson| KS|67410|984-226-9841|884-246-9441|adell.christiane@...|http://www.magnus...| -1|
 1238| Adell| Jovita| Centro Inc|1238 Acuff St. Ku...| New York| New York| NY|10013|984-226-9991|884-246-9591|adell.jovita@gmai...|http://www.centro...| -1|
 1342| Adell| Micaela|Warehouse Office ...|1342 Acuff St. Pa...| Elkhart| Elkhart| IN|46514|984-227-0095|884-246-9695|adell.micaela@gma...|http://www.wareho...| -1|
 1580| Ahmed| Chantell| Johnson, Wes Esq|1580 Adkin St. Coody| Round Rock| Williamson| TX|78664|984-227-0333|884-246-9933|ahmed.chantell@gm...|http://www.johnso...| -1|
 1591| Ahmed| Clorinda|Students In Free ...|1591 Adkin St. Cryer| Milwaukee| Milwaukee| WI|53226|984-227-0344|884-246-9944|ahmed.clorinda@gm...|http://www.studen...| -1|
 1645| Ahmed| Estrella|Haavisto, Brian F...|1645 Adkin St. Fu...| Alliance| Box Butte| NE|69301|984-227-0398|884-246-9998|ahmed.estrella@gm...|http://www.haavis...| -1|
 1829| Ahmed| Martina| Chadds Ford Winery| 1829 Adkin St. Paa| Indianapolis| Marion| IN|46202|984-227-0582|884-247-0182|ahmed.martina@gma...|http://www.chadds...| -1|
 1959| Ahmed| Tonette|Galaxy Internatio...|1959 Adkin St. Va...| Mc Minnville| Warren| TN|37110|984-227-0712|884-247-0312|ahmed.tonette@gma...|http://www.galaxy...| -1|
 2122| Aja| Donte| Orinda News|2122 Agramonte St...| Middle Island| Suffolk| NY|11953|984-227-0875|884-247-0475| aja.donte@gmail.com|http://www.orinda...| -1|
 2142| Aja| Ernest|H H H Enterprises...|2142 Agramonte St...| Miami| Miami-Dade| FL|33155|984-227-0895|884-247-0495|aja.ernest@gmail.com|http://www.hhhent...| -1|
 2366| Aja| Olive|Royal Pontiac Old...|2366 Agramonte St...| Green Bay| Brown| WI|54301|984-227-1119|884-247-0719| aja.olive@gmail.com|http://www.royalp...| -1|
 2659| Alaine| Francine|Farmers Insurance...|2659 Ahle St. Gol...| Hamilton| Butler| OH|45011|984-227-1412|884-247-1012|alaine.francine@g...|http://www.farmer...| -1|
 2866| Alaine| Oretha| Commercial Press| 2866 Ahle St. Riden| Bangor| Penobscot| ME|04401|984-227-1619|884-247-1219|alaine.oretha@gma...|http://www.commer...| -1|
 3175| Albina| Glendora|Kummerer, J Micha...|3175 Albares St. ...| San Gabriel|Los Angeles| CA|91776|984-227-1928|884-247-1528|albina.glendora@g...|http://www.kummer...| -1|
 3749| Alease| Kattie| Cobb, James O Esq|3749 Amigon St. L...| Syracuse| Onondaga| NY|13214|984-227-2502|884-247-2102|alease.kattie@gma...|http://www.cobbja...| -1|
+------+----------+----------+--------------------+--------------------+---------------+-----------+-----+-----+------------+------------+--------------------+--------------------+--------+
only showing top 20 rows

In [0]:
VaccinationSequence=VaccinationDrivePlan.join(CityEmployeeDensity,CityEmployeeDensity.city==VaccinationDrivePlan.city,"left")\
                           .select(VaccinationDrivePlan.emp_id,
                                  VaccinationDrivePlan.first_name,
                                  VaccinationDrivePlan.last_name,
                                  VaccinationDrivePlan.company_name,
                                  VaccinationDrivePlan.address,
                                  VaccinationDrivePlan.city,
                                  VaccinationDrivePlan.county,
                                  VaccinationDrivePlan.state,
                                  VaccinationDrivePlan.zip,
                                  VaccinationDrivePlan.phone1,
                                  VaccinationDrivePlan.phone2,
                                  VaccinationDrivePlan.email,
                                  VaccinationDrivePlan.web,
                                  CityEmployeeDensity.vaccination_order.alias('sequence'))
VaccinationSequence.show()
VaccinationSequence.persist()
VaccinationSequence.registerTempTable("VaccinationSequence")



+------+----------+----------+--------------------+--------------------+---------------+-----------+-----+-----+------------+------------+--------------------+--------------------+--------+
emp_id|first_name| last_name| company_name| address| city| county|state| zip| phone1| phone2| email| web|sequence|
+------+----------+----------+--------------------+--------------------+---------------+-----------+-----+-----+------------+------------+--------------------+--------------------+--------+
 148| Ernie| Stenseth| Knwz Newsradio| 45 E Liberty St|Ridgefield Park| Bergen| NJ|07660|201-709-6245|201-387-9093|ernie_stenseth@ao...|http://www.knwzne...| 103|
 463| Tonette| Wenner|Northwest Publishing| 4545 Courthouse Rd| Westbury| Nassau| NY|11590|516-968-6051|516-333-4861| twenner@aol.com|http://www.northw...| 94|
 471| Valentin| Klimek|Schmid, Gayanne K...| 137 Pioneer Way| Chicago| Cook| IL|60604|312-303-5453|312-512-2338| vklimek@klimek.org|http://www.schmid...| 3|
 496| Yolando| Luczki|Dal Tile Corporation| 422 E 21st St| Syracuse| Onondaga| NY|13214|315-304-4759|315-640-6357| yolando@cox.net|http://www.daltil...| 101|
 833| Adelina| Maryann| Kleensteel|833 Acey St. Pagl...| Amarillo| Randall| TX|79109|984-226-9586|884-246-9186|adelina.maryann@g...|http://www.kleens...| 76|
 1088| Adell|Christiane| Magnuson|1088 Acuff St. Co...| Abilene| Dickinson| KS|67410|984-226-9841|884-246-9441|adell.christiane@...|http://www.magnus...| 21|
 1238| Adell| Jovita| Centro Inc|1238 Acuff St. Ku...| New York| New York| NY|10013|984-226-9991|884-246-9591|adell.jovita@gmai...|http://www.centro...| 1|
 1342| Adell| Micaela|Warehouse Office ...|1342 Acuff St. Pa...| Elkhart| Elkhart| IN|46514|984-227-0095|884-246-9695|adell.micaela@gma...|http://www.wareho...| 48|
 1580| Ahmed| Chantell| Johnson, Wes Esq|1580 Adkin St. Coody| Round Rock| Williamson| TX|78664|984-227-0333|884-246-9933|ahmed.chantell@gm...|http://www.johnso...| 92|
 1591| Ahmed| Clorinda|Students In Free ...|1591 Adkin St. Cryer| Milwaukee| Milwaukee| WI|53226|984-227-0344|884-246-9944|ahmed.clorinda@gm...|http://www.studen...| 6|
 1645| Ahmed| Estrella|Haavisto, Brian F...|1645 Adkin St. Fu...| Alliance| Box Butte| NE|69301|984-227-0398|884-246-9998|ahmed.estrella@gm...|http://www.haavis...| 99|
 1829| Ahmed| Martina| Chadds Ford Winery| 1829 Adkin St. Paa| Indianapolis| Marion| IN|46202|984-227-0582|884-247-0182|ahmed.martina@gma...|http://www.chadds...| 9|
 1959| Ahmed| Tonette|Galaxy Internatio...|1959 Adkin St. Va...| Mc Minnville| Warren| TN|37110|984-227-0712|884-247-0312|ahmed.tonette@gma...|http://www.galaxy...| 86|
 2122| Aja| Donte| Orinda News|2122 Agramonte St...| Middle Island| Suffolk| NY|11953|984-227-0875|884-247-0475| aja.donte@gmail.com|http://www.orinda...| 54|
 2142| Aja| Ernest|H H H Enterprises...|2142 Agramonte St...| Miami| Miami-Dade| FL|33155|984-227-0895|884-247-0495|aja.ernest@gmail.com|http://www.hhhent...| 4|
 2366| Aja| Olive|Royal Pontiac Old...|2366 Agramonte St...| Green Bay| Brown| WI|54301|984-227-1119|884-247-0719| aja.olive@gmail.com|http://www.royalp...| 95|
 2659| Alaine| Francine|Farmers Insurance...|2659 Ahle St. Gol...| Hamilton| Butler| OH|45011|984-227-1412|884-247-1012|alaine.francine@g...|http://www.farmer...| 90|
 2866| Alaine| Oretha| Commercial Press| 2866 Ahle St. Riden| Bangor| Penobscot| ME|04401|984-227-1619|884-247-1219|alaine.oretha@gma...|http://www.commer...| 88|
 3175| Albina| Glendora|Kummerer, J Micha...|3175 Albares St. ...| San Gabriel|Los Angeles| CA|91776|984-227-1928|884-247-1528|albina.glendora@g...|http://www.kummer...| 98|
 3749| Alease| Kattie| Cobb, James O Esq|3749 Amigon St. L...| Syracuse| Onondaga| NY|13214|984-227-2502|884-247-2102|alease.kattie@gma...|http://www.cobbja...| 101|
+------+----------+----------+--------------------+--------------------+---------------+-----------+-----+-----+------------+------------+--------------------+--------------------+--------+
only showing top 20 rows

In [0]:
VaccinationSchedule=spark.sql("""with vacc_order as (
                                    SELECT emp_id,city,row_number() over (partition by city order by emp_id) as position
                                    FROM VaccinationSequence
                                )
                                SELECT emp_id,city,floor(((position-1)/100))+1 as day
                                FROM vacc_order
                                order by 2,3,1
                                """)
VaccinationSchedule.persist()
VaccinationSchedule.show()
VaccinationSchedule.registerTempTable("VaccinationSchedule")

+------+--------+---+
emp_id| city|day|
+------+--------+---+
 119|Aberdeen| 1|
 594|Aberdeen| 1|
 660|Aberdeen| 1|
 704|Aberdeen| 1|
 902|Aberdeen| 1|
 1353|Aberdeen| 1|
 1463|Aberdeen| 1|
 1804|Aberdeen| 1|
 1826|Aberdeen| 1|
 1881|Aberdeen| 1|
 1886|Aberdeen| 1|
 1903|Aberdeen| 1|
 1927|Aberdeen| 1|
 2134|Aberdeen| 1|
 2206|Aberdeen| 1|
 2233|Aberdeen| 1|
 2255|Aberdeen| 1|
 2537|Aberdeen| 1|
 2690|Aberdeen| 1|
 2706|Aberdeen| 1|
+------+--------+---+
only showing top 20 rows

In [0]:
VaccinationScheduleSeq=spark.sql("""
                                    with vacc_order as (
                                    SELECT emp_id,city,row_number() over (order by sequence,emp_id) as position
                                    FROM VaccinationSequence
                                )
                                SELECT emp_id,city,floor(((position-1)/100))+1 as day
                                FROM vacc_order
                                order by 3,2,1
                                 """)
VaccinationScheduleSeq.persist()
VaccinationScheduleSeq.show()
VaccinationScheduleSeq.registerTempTable("VaccinationScheduleSeq")

+------+--------+---+
emp_id| city|day|
+------+--------+---+
 13|New York| 1|
 58|New York| 1|
 103|New York| 1|
 118|New York| 1|
 157|New York| 1|
 194|New York| 1|
 225|New York| 1|
 237|New York| 1|
 246|New York| 1|
 287|New York| 1|
 348|New York| 1|
 373|New York| 1|
 451|New York| 1|
 490|New York| 1|
 529|New York| 1|
 545|New York| 1|
 589|New York| 1|
 649|New York| 1|
 663|New York| 1|
 701|New York| 1|
+------+--------+---+
only showing top 20 rows

In [0]:
estimatedCompletion=spark.sql("""SELECT city,MAX(day) as Reqd_days
FROM VaccinationSchedule
GROUP BY city
ORDER BY 2 DESC""")

estimatedCompletion.persist()
estimatedCompletion.show()
estimatedCompletion.registerTempTable("estimatedCompletion")



+-------------+---------+
 city|Reqd_days|
+-------------+---------+
 New York| 11|
 Philadelphia| 9|
 Chicago| 8|
 Miami| 7|
 Orlando| 7|
 Milwaukee| 6|
 Providence| 6|
 Los Angeles| 5|
 Indianapolis| 5|
 Aberdeen| 5|
 Atlanta| 5|
 Brooklyn| 5|
 Baltimore| 5|
 Denver| 5|
 Phoenix| 4|
 San Diego| 4|
San Francisco| 4|
 Bridgeport| 4|
 Boise| 4|
 New Orleans| 4|
+-------------+---------+
only showing top 20 rows

In [0]:
%sql

 
-- SELECT "Please run all the above cells and query the datasets here"
SELECT * FROM employees

emp_id,first_name,last_name,company_name,address,city,county,state,zip,phone1,phone2,email,web
148,Ernie,Stenseth,Knwz Newsradio,45 E Liberty St,Ridgefield Park,Bergen,NJ,07660,201-709-6245,201-387-9093,ernie_stenseth@aol.com,http://www.knwznewsradio.com
463,Tonette,Wenner,Northwest Publishing,4545 Courthouse Rd,Westbury,Nassau,NY,11590,516-968-6051,516-333-4861,twenner@aol.com,http://www.northwestpublishing.com
471,Valentin,Klimek,"Schmid, Gayanne K Esq",137 Pioneer Way,Chicago,Cook,IL,60604,312-303-5453,312-512-2338,vklimek@klimek.org,http://www.schmidgayannekesq.com
496,Yolando,Luczki,Dal Tile Corporation,422 E 21st St,Syracuse,Onondaga,NY,13214,315-304-4759,315-640-6357,yolando@cox.net,http://www.daltilecorporation.com
833,Adelina,Maryann,Kleensteel,833 Acey St. Pagliuca,Amarillo,Randall,TX,79109,984-226-9586,884-246-9186,adelina.maryann@gmail.com,http://www.kleensteel.com
1088,Adell,Christiane,Magnuson,1088 Acuff St. Coyier,Abilene,Dickinson,KS,67410,984-226-9841,884-246-9441,adell.christiane@gmail.com,http://www.magnuson.com
1238,Adell,Jovita,Centro Inc,1238 Acuff St. Kulzer,New York,New York,NY,10013,984-226-9991,884-246-9591,adell.jovita@gmail.com,http://www.centroinc.com
1342,Adell,Micaela,Warehouse Office & Paper Prod,1342 Acuff St. Pawlowicz,Elkhart,Elkhart,IN,46514,984-227-0095,884-246-9695,adell.micaela@gmail.com,http://www.warehouseofficepaperprod.com
1580,Ahmed,Chantell,"Johnson, Wes Esq",1580 Adkin St. Coody,Round Rock,Williamson,TX,78664,984-227-0333,884-246-9933,ahmed.chantell@gmail.com,http://www.johnsonwesesq.com
1591,Ahmed,Clorinda,Students In Free Entrprs Natl,1591 Adkin St. Cryer,Milwaukee,Milwaukee,WI,53226,984-227-0344,884-246-9944,ahmed.clorinda@gmail.com,http://www.studentsinfreeentrprsnatl.com


In [0]:
%sh

ls -l

total 156
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
-rw-r--r-- 1 root root 1414 Apr 3 12:40 derby.log
drwxr-xr-x 3 root root 4096 Apr 3 12:39 eventlogs
drwxr-xr-x 2 root root 4096 Apr 3 12:45 ganglia
drwxr-xr-x 2 root root 4096 Apr 3 12:39 logs
drwxr-xr-x 5 root root 4096 Apr 3 12:40 metastore_db
-rw-r--r-- 1 root root 92320 Apr 3 12:48 us-500.csv
-rw-r--r-- 1 root root 38029 Apr 3 12:48 us-500.zip
-rw-r--r-- 1 root root 0 Apr 3 12:45 us-500.zipnif7emqj.tmp
-rw-r--r-- 1 root root 0 Apr 3 12:45 us-500.zipqu22kz_e.tmp